Here are the peak calls for the snATAC-seq data (excluding biobank samples) by cell type:
- cytokine+untreated:/nfs/lab/elisha/islets/cytokine/new_islet/gaulton/peaks/gatsby/cellType_cytunt
- cytokine: /nfs/lab/elisha/islets/cytokine/new_islet/gaulton/peaks/gatsby/cellType_cyt
- untreated: /nfs/lab/elisha/islets/cytokine/new_islet/gaulton/peaks/gatsby/cellType_unt 

In [1]:
library(stringr)
library(RColorBrewer)

In [2]:
setwd("/nfs/lab/projects/islet_cytok/analysis/snATAC/peaks")

In [3]:
annp = read.table('Islet_bulkATAC.subpop.sorted.merged.bed')

In [4]:
annp$counts = str_count(annp$V4,",")+1
annp$peaks = paste("peak", annp[,1], annp[,2], annp[,3], sep="-")

In [5]:
u = read.table("../../deseq_ATAC_NEW_bedfiles/annotated_peaks.tsv")

In [6]:
annp$upr = annp$peaks %in% rownames(u) [rowSums(u[,c(2,4,6,8,10,12,14)])>0]
annp$dor = annp$peaks %in% rownames(u) [rowSums(u[,c(1,3,5,7,9,11,13)])>0]

In [7]:
sum(rowSums(u)>0)

[1] 44969

In [9]:
head(annp)

,V1,V2,V3,V4,counts,peaks,upr,dor
,<fct>,<int>,<int>,<fct>,<dbl>,<chr>,<lgl>,<lgl>
1,chr1,755248,757745,beta,1,peak-chr1-755248-757745,FALSE,FALSE
2,chr1,762279,763259,"acinar,alpha,beta,delta,endothelial,stellate",6,peak-chr1-762279-763259,FALSE,FALSE
3,chr1,767391,768426,"acinar,alpha,beta,delta,stellate",5,peak-chr1-767391-768426,FALSE,TRUE
4,chr1,786797,787798,"alpha,beta",2,peak-chr1-786797-787798,FALSE,FALSE
5,chr1,793418,794514,"acinar,alpha,beta,delta,stellate",5,peak-chr1-793418-794514,FALSE,TRUE
6,chr1,804948,805700,"acinar,alpha,beta,delta,endothelial,stellate",6,peak-chr1-804948-805700,FALSE,FALSE


In [20]:
bed1 = read.table('Islet_bulkATAC.bed' )
bed2 = read.table('Islet_bulkATAC.subpop.sorted.merged.bed')

In [39]:
bed1$V4="z"
bedm = rbind(bed1, bed2)

In [40]:
bedm = bedm[order(bedm$V4),]

In [41]:
bedm = bedm[!duplicated(bedm[,1:3]),]

In [42]:
bedm$V4[bedm$V4=='z']<-"na"

In [43]:
peaks = paste("peak", bedm[,1], bedm[,2], bedm[,3], sep="-")

In [50]:
sum(duplicated(annp$peaks))

[1] 0

In [54]:
dim(bedm)

[1] 165884      5

In [44]:
bedm$cyt = peaks %in% rownames(u)

In [46]:
tab1 = subset(bedm, grepl("beta", bedm$V4))

In [55]:
colnames(bedm) = c("chrom", "start", "end", "islet_cell_type", "cytokine_responsive")

In [56]:
write.table(bedm, "cCREs_annotated.bed", sep="\t", quote=F, row.names=F)

In [7]:
cells = c('acinar','alpha','beta','delta','endothelial','stellate')
mat1 =  matrix(NA, nrow=2, ncol=6)
colnames(mat1) = cells
rownames(mat1) = c("down", "up")
mat2 = mat1
for (cell in cells ){
    tab1 = subset(annp, grepl(cell, annp$V4))
    write.table(tab1[tab1$upr==T,1:3], paste0("/nfs/lab/projects/islet_cytok/analysis/ldsc/", cell, ".up.bed"), 
                sep="\t", quote=F, row.names=F, col.names=F)
    write.table(tab1[tab1$dor==T,1:3], paste0("/nfs/lab/projects/islet_cytok/analysis/ldsc/", cell, ".down.bed"), 
                sep="\t", quote=F, row.names=F, col.names=F)
    mat1["up",cell] = sum(tab1$upr)
    mat1["down",cell] = sum(tab1$dor)
    
    tab2 = subset(annp, annp$V4 == cell)
    mat2["up",cell] = sum(tab2$upr)
    mat2["down",cell] = sum(tab2$dor)
}

In [13]:
mat1
colSums(mat1)

,acinar,alpha,beta,delta,endothelial,stellate
down,9441,19447,20097,15951,4554,13257
up,10880,16648,18881,12990,4840,12610


acinar       alpha        beta       delta endothelial    stellate 
      20321       36095       38978       28941        9394       25867

In [15]:
3186/38978

[1] 0.08173842

In [14]:
mat2
colSums(mat2)

,acinar,alpha,beta,delta,endothelial,stellate
down,14,962,1314,63,1,20
up,239,670,1872,128,20,390


acinar       alpha        beta       delta endothelial    stellate 
        253        1632        3186         191          21         410

In [67]:
pdf("../../cytokine_figures/snatac_annotated_peaks.pdf")
par(mfrow=c(2,2),mar = c(4,4,4,5))
b = barplot(mat1, las=3,
           ylim=c(0,50000), col = brewer.pal(9, "PuOr")[c(3,7)] )

text(x = b, y =mat1[1,], labels = paste0(round(unlist(mat1[1 ,])/1000,1),"k") , pos = 1, cex=0.75)
text(x = b, y = as.numeric(mat1[2,])+ as.numeric(mat1[1,]), 
     labels = paste0(round(unlist(mat1[2,])/1000,1),"k") , pos = 1, cex=0.75)

b = barplot(mat2, las=3,
           ylim=c(0,3500), col = brewer.pal(9, "PuOr")[c(3,7)] )

text(x = b, y =mat2[1,], labels = paste0(round(unlist(mat2[1 ,])/1000,1),"k") , pos = 1, cex=0.75)
text(x = b, y = as.numeric(mat2[2,])+ as.numeric(mat2[1,]), 
     labels = paste0(round(unlist(mat2[2,])/1000,1),"k") , pos = 1, cex=0.75)
dev.off()

png 
  2

In [55]:
 tab2 = subset(annp, V4 == "beta" & upr == TRUE)

In [56]:
head(tab2)

,V1,V2,V3,V4,counts,peaks,upr,dor
,<fct>,<int>,<int>,<fct>,<dbl>,<chr>,<lgl>,<lgl>
73,chr1,1353266,1353689,beta,1,peak-chr1-1353266-1353689,TRUE,FALSE
106,chr1,1720184,1720938,beta,1,peak-chr1-1720184-1720938,TRUE,FALSE
270,chr1,3807705,3807970,beta,1,peak-chr1-3807705-3807970,TRUE,FALSE
590,chr1,8593207,8593540,beta,1,peak-chr1-8593207-8593540,TRUE,FALSE
1376,chr1,19496023,19496554,beta,1,peak-chr1-19496023-19496554,TRUE,FALSE
1426,chr1,19884316,19884604,beta,1,peak-chr1-19884316-19884604,TRUE,FALSE


In [57]:
cyt = read.table('/nfs/lab/elisha/islets/cytokine/update_deseq/output/cytokine_24hr_Hi.dec16.filtered.txt')
cyt = cyt[order(cyt$pvalue),]

In [59]:
cytb = subset(cyt, rownames(cyt) %in% tab2$peaks)

In [61]:
head(cytb,20)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
peak-chr4-17581932-17582182,100.31767,3.889667,0.2597773,14.973084,1.100953e-50,6.087683e-46
peak-chr6-43804096-43804441,172.62966,1.721841,0.1625232,10.594432,3.162551e-26,3.544706e-23
peak-chr3-115775143-115775361,51.70124,3.245185,0.3372836,9.621530,6.485925e-22,3.610440e-19
peak-chr2-133178441-133178662,78.75060,2.464606,0.2639840,9.336194,9.985890e-21,4.575965e-18
peak-chr3-71772786-71773061,55.71386,2.724314,0.2973036,9.163408,5.028382e-20,2.000307e-17
peak-chr3-179984965-179985522,56.59627,2.944213,0.3508117,8.392574,4.756142e-17,1.053362e-14
peak-chr20-56810012-56810439,58.07287,2.961955,0.3530934,8.388589,4.920095e-17,1.086771e-14
peak-chr1-54875176-54875462,93.82417,1.973000,0.2362314,8.351979,6.712852e-17,1.440562e-14
peak-chr7-154180526-154180932,44.27482,2.945653,0.3532528,8.338654,7.514078e-17,1.589879e-14


In [9]:
head(annp)

,V1,V2,V3,V4,counts,peaks,upr,dor
,<fct>,<int>,<int>,<fct>,<dbl>,<chr>,<lgl>,<lgl>
1,chr1,755248,757745,beta,1,peak-chr1-755248-757745,FALSE,FALSE
2,chr1,762279,763259,"acinar,alpha,beta,delta,endothelial,stellate",6,peak-chr1-762279-763259,FALSE,FALSE
3,chr1,767391,768426,"acinar,alpha,beta,delta,stellate",5,peak-chr1-767391-768426,FALSE,TRUE
4,chr1,786797,787798,"alpha,beta",2,peak-chr1-786797-787798,FALSE,FALSE
5,chr1,793418,794514,"acinar,alpha,beta,delta,stellate",5,peak-chr1-793418-794514,FALSE,TRUE
6,chr1,804948,805700,"acinar,alpha,beta,delta,endothelial,stellate",6,peak-chr1-804948-805700,FALSE,FALSE


## Motif enrichment in beta cell specific peaks 

In [11]:
tab1 = subset(annp, V4=="beta")
    write.table(tab1[tab1$upr==T,1:3],  "homer_beta/beta_spec.up.bed", 
                sep="\t", quote=F, row.names=F, col.names=F)
    write.table(tab1[tab1$dor==T,1:3],  "homer_beta/beta_spec.down.bed", 
                sep="\t", quote=F, row.names=F, col.names=F)

In [14]:
write.table(annp[annp$upr==T,1:3],  "homer_beta/all.up.bed", 
                sep="\t", quote=F, row.names=F, col.names=F)
    write.table(annp[annp$dor==T,1:3],  "homer_beta/all.down.bed", 
                sep="\t", quote=F, row.names=F, col.names=F)

In [13]:
run_homer = function(bedfile, outdir, bgfile )
{
    system(paste ('findMotifsGenome.pl', bedfile, '/nfs/lab/KG/ref/male.masked.hg19.fa', 
       outdir,  '-bg', bgfile,  '-nomotif' ))
}

In [15]:
run_homer(bedfile = paste0(getwd(),"/homer_beta/beta_spec.up.bed"),
          outdir  = paste0(getwd(),"/homer_beta/beta_spec.up"),
          bgfile  = paste0(getwd(),"/homer_beta/all.up.bed"))

In [ ]:
run_homer(bedfile = paste0(getwd(),"/homer_beta/beta_spec.down.bed"),
          outdir  = paste0(getwd(),"/homer_beta/beta_spec.down"),
          bgfile  = paste0(getwd(),"/homer_beta/all.down.bed"))